In [1]:
import argparse
import time
import csv
import pickle
import operator
import datetime
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import LabelEncoder

In [20]:
parser = argparse.ArgumentParser()
parser.add_argument('--dataset', default='sample', help='dataset name: ml1m/lfm1b/sample')
opt, unknown = parser.parse_known_args()
print(opt.dataset)
dataset = 'Tmall.csv'

sample


In [21]:
with open(dataset, "r") as f:
    reader = csv.DictReader(f)
    sess_clicks = {}
    sess_date = {}
    ctr = 0
    curid = -1
    curdate = None
    for data in reader:
        sessid = int(data['SessionId'])
        if curdate and not curid == sessid:
            #date = curdate
            date = float(data['timestamp'])
            sess_date[curid] = date
        curid = sessid
        item = int(data['ItemId'])
        curdate = float(data['Time'])

        if sessid in sess_clicks:
            sess_clicks[sessid] += [item]
        else:
            sess_clicks[sessid] = [item]
        ctr += 1
    date = float(data['Time'])
    sess_date[curid] = date
print("-- Reading data @ %ss" % datetime.datetime.now())

# Filter out length 1 sessions
for s in list(sess_clicks):
    if len(sess_clicks[s]) == 1:
        del sess_clicks[s]
        del sess_date[s]

# Count number of times each item appears
iid_counts = {}
for s in sess_clicks:
    seq = sess_clicks[s]
    for iid in seq:
        if iid in iid_counts:
            iid_counts[iid] += 1
        else:
            iid_counts[iid] = 1

sorted_counts = sorted(iid_counts.items(), key=operator.itemgetter(1))

length = len(sess_clicks)
for s in list(sess_clicks):
    curseq = sess_clicks[s]
    filseq = list(filter(lambda i: iid_counts[i] >= 5, curseq))
    if len(filseq) < 2 or len(filseq) > 40:
        del sess_clicks[s]
        del sess_date[s]
    else:
        sess_clicks[s] = filseq

dates = list(sess_date.items())
#maxdate = dates[0][1]


#for _, date in dates:
#if maxdate < date:
#maxdate = date

#maxdate = 998315055 這是movielen1M
maxdate = 1447196533
splitdate = 0

#splitdate = maxdate - 86400 * 300   # the number of seconds for a day：86400
splitdate = maxdate - 100

print('Splitting date', splitdate)  
tra_sess = list(filter(lambda x: int(x[1]) < splitdate, dates))
tes_sess = list(filter(lambda x: int(x[1]) > splitdate, dates))

tra_sess = sorted(tra_sess, key=operator.itemgetter(1))  # [(session_id, timestamp), (), ]
tes_sess = sorted(tes_sess, key=operator.itemgetter(1))  # [(session_id, timestamp), (), ]
print(len(tra_sess))  # 186670    # 7966257
print(len(tes_sess))  # 15979     # 15324
print(tra_sess[:3])
print(tes_sess[:3])
#print("-- Splitting train set and test set @ %ss" % datetime.datetime.now())

item_dict = {}


# Convert training sessions to sequences and renumber items to start from 1
def obtian_tra():
    train_ids = []
    train_seqs = []
    train_dates = []
    item_ctr = 1
    for s, date in tra_sess:
        seq = sess_clicks[s]
        outseq = []
        for i in seq:
            if i in item_dict:
                outseq += [i]
            else:
                outseq += [i]
                item_dict[i] = i
                item_ctr += 1
        if len(outseq) < 2:  # Doesn't occur
            continue
        train_ids += [s]
        train_dates += [date]
        train_seqs += [outseq]
    print('item_ctr')
    print(item_ctr)     # 43098, 37484
    return train_ids, train_dates, train_seqs

# Convert test sessions to sequences, ignoring items that do not appear in training set
def obtian_tes():
    test_ids = []
    test_seqs = []
    test_dates = []
    for s, date in tes_sess:
        seq = sess_clicks[s]
        outseq = []
        for i in seq:
            if i in item_dict:
                outseq += [i]
        if len(outseq) < 2:
            continue
        test_ids += [s]
        test_dates += [date]
        test_seqs += [outseq]
    return test_ids, test_dates, test_seqs

tra_ids, tra_dates, tra_seqs = obtian_tra()
tes_ids, tes_dates, tes_seqs = obtian_tes()

def process_seqs(iseqs, idates):
    out_seqs = []
    out_dates = []
    labs = []
    ids = []
    for id, seq, date in zip(range(len(iseqs)), iseqs, idates):
        for i in range(1, len(seq)):
            tar = seq[-i]
            labs += [tar]
            out_seqs += [seq[:-i]]
            out_dates += [date]
            ids += [id]
    return out_seqs, out_dates, labs, ids

tr_seqs, tr_dates, tr_labs, tr_ids = process_seqs(tra_seqs, tra_dates)
te_seqs, te_dates, te_labs, te_ids = process_seqs(tes_seqs, tes_dates)
tra = (tr_seqs, tr_labs)
tes = (te_seqs, te_labs)
print(len(tr_seqs))
print(len(te_seqs))
print(tr_seqs[:3], tr_dates[:3], tr_labs[:3])
print(te_seqs[:3], te_dates[:3], te_labs[:3])
all = 0

for seq in tra_seqs:
    all += len(seq)
for seq in tes_seqs:
    all += len(seq)
print('avg length: ', all * 1.0/(len(tra_seqs) + len(tes_seqs)))

if not os.path.exists('tmall'):
    os.makedirs('tmall')
pickle.dump(tra, open('tmall/train.txt', 'wb'))
pickle.dump(tes, open('tmall/test.txt', 'wb'))
pickle.dump(tra_seqs, open('tmall/all_train_seq.txt', 'wb'))

print('Done.')

Splitting date 1447196433
64811
998
[('976', 1432072801.0), ('1064', 1432072801.0), ('1227', 1432072801.0)]
[('1226', 1447196434.0), ('2940', 1447196434.0), ('3583', 1447196434.0)]
item_ctr
39762
348766
25412
[[21929], [24275], [30270]] [1432072801.0, 1432072801.0, 1432072801.0] [16059, 16501, 36301]
[[35758, 24830, 20314, 28665, 4048, 16595, 11813, 24830, 30356, 15961, 15961, 36342, 25495, 15961], [35758, 24830, 20314, 28665, 4048, 16595, 11813, 24830, 30356, 15961, 15961, 36342, 25495], [35758, 24830, 20314, 28665, 4048, 16595, 11813, 24830, 30356, 15961, 15961, 36342]] [1447196434.0, 1447196434.0, 1447196434.0] [28665, 15961, 25495]
avg length:  6.685818049202997
Done.


# 數據前處裡

In [172]:
import pickle

with open('movielens/test.txt', 'rb') as t:
    foo_list = pickle.load(t)

In [237]:
import numpy as np
import pandas as pd

dataset = pd.read_csv("./dataset/ml-1m/ratings.txt", sep="::", header=None, names=["UserId", "movieID","rating","timestamp"])
dataset

C:\Users\c8905\AppData\Local\Temp\ipykernel_22768\4288598303.py:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  dataset = pd.read_csv("./dataset/ml-1m/ratings.txt", sep="::", header=None, names=["UserId", "movieID","rating","timestamp"])


,UserId,movieID,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


In [261]:
dataset = dataset.groupby('UserId').apply(lambda x: x.sort_values('timestamp', ascending=True))
dataset

UserId  movieID  rating  timestamp
UserId                                            
1      31            1     3186       4  978300019
       27            1     1721       4  978300055
       37            1     1022       5  978300055
       22            1     1270       5  978300055
       24            1     2340       3  978300103
...                ...      ...     ...        ...
6040   1000019    6040     2917       4  997454429
       1000172    6040     1784       3  997454464
       999988     6040     1921       4  997454464
       1000167    6040      161       3  997454486
       1000042    6040     1221       4  998315055

[1000209 rows x 4 columns]

In [263]:
dataset = dataset.droplevel('UserId')

In [273]:
dataset = dataset.reset_index(level=None, drop=True, inplace=False, col_level=0,col_fill='')

In [274]:
from datetime import datetime

date = datetime.fromtimestamp(dataset.iloc[0,3])
dataset.iloc[0,3]

978300019

## timestamp如果不是在同一天就直接換另一個sessionID

In [72]:
session_num = 0
now_user = 1
now_date = datetime.fromtimestamp(dataset.iloc[0,3]).date()
session_lis = []
for index, row in dataset.iterrows():
    if row['UserId'] == now_user :
        if datetime.fromtimestamp(row['timestamp']).date() == now_date:
            session_lis.append(session_num)
        else:
            session_num +=1
            session_lis.append(session_num)
            now_date = datetime.fromtimestamp(row['timestamp']).date()
    else:
        now_user += 1
        now_date = datetime.fromtimestamp(row['timestamp']).date()
        session_num += 1
        if datetime.fromtimestamp(row['timestamp']).date() == now_date:
            session_lis.append(session_num)
        else:
            session_num +=1
            session_lis.append(session_num)

KeyboardInterrupt: 

## 這邊這個是按照session開始往後推一天

In [265]:
session_num = 0
now_user = 1
##978386419是一天
spilt_date = dataset.iloc[0,3]+600
session_lis = []
for index, row in dataset.iterrows():
    if row['UserId'] == now_user :
        if row['timestamp'] < spilt_date:
            session_lis.append(session_num)
        else:
            session_num +=1
            session_lis.append(session_num)
            spilt_date = row['timestamp']+600
    else:
        now_user += 1
        spilt_date = row['timestamp']+600
        session_num += 1
        if row['timestamp'] < spilt_date:
            session_lis.append(session_num)
        else:
            session_num +=1
            session_lis.append(session_num)

In [268]:
session_lis

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 1

In [275]:
sorted_movielens = pd.concat([dataset,pd.DataFrame(session_lis,columns=['sessionID'])],axis=1)
#sorted_movielens = sorted_movielens.groupby('UserId').apply(lambda x: x.sort_values('timestamp', ascending=True))

In [276]:
sorted_movielens

,UserId,movieID,rating,timestamp,sessionID
0,1,3186,4,978300019,0
1,1,1721,4,978300055,0
2,1,1022,5,978300055,0
3,1,1270,5,978300055,0
4,1,2340,3,978300103,0
...,...,...,...,...,...
1000204,6040,2917,4,997454429,54833
1000205,6040,1784,3,997454464,54833
1000206,6040,1921,4,997454464,54833
1000207,6040,161,3,997454486,54833


## 現在的movieId是從1開始，但是圖譜是從0開始，所以要用json檔轉換

In [277]:
import json 

with open('./dataset/ml-1m/trainning-link/eid2index.json') as f:
    data = json.load(f)

movie_set = set([])
for index, row in sorted_movielens.iterrows():
    if str(row['movieID']) in data.keys():
        sorted_movielens.loc[index, 'movieID'] = data[str(row['movieID'])]+1
        movie_set.add(row['movieID'])
    else:
        sorted_movielens = sorted_movielens.drop(index=[index])
movie_set

{1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 110,
 111,
 112,
 113,
 114,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185,
 186,
 187,
 188,
 189,
 190,
 1

In [278]:
sorted_movielens

,UserId,movieID,rating,timestamp,sessionID
0,1,1234,4,978300019,0
1,1,1660,4,978300055,0
2,1,1040,5,978300055,0
3,1,2655,5,978300055,0
4,1,845,3,978300103,0
...,...,...,...,...,...
1000204,6040,3080,4,997454429,54833
1000205,6040,3187,3,997454464,54833
1000206,6040,3273,4,997454464,54833
1000207,6040,3539,3,997454486,54833


## 要將有出現在知識圖譜中，但沒有出現在推薦系統中的id拿掉

In [95]:
##但表小於等於3806的都是item，大於的都是entity
item_id = list(movie_set)[-1]

In [96]:
kg_path = './kg.txt'
lines = []
with open(kg_path, 'r') as f:
    for line in f.readlines():
        # if line.strip() == '':
        #     continue

        arr = line.strip().split('\t')
        # arr = line.strip().split(' ')
        head, rel, tail = int(arr[0]), arr[1], int(arr[2])
        if head <= item_id and head in movie_set:
            if tail <= item_id and tail in movie_set:
                lines.append(line)
            elif tail > item_id:
                lines.append(line)
        elif head > item_id:
            if tail <= item_id and tail in movie_set:
                lines.append(line)
            elif tail > item_id:
                lines.append(line)

In [97]:
lines

['1925\t3\t36068\n',
 '36068\t6\t1925\n',
 '1925\t3\t10999\n',
 '10999\t6\t1925\n',
 '1925\t3\t65992\n',
 '65992\t6\t1925\n',
 '1925\t3\t138199\n',
 '138199\t6\t1925\n',
 '1925\t3\t4567\n',
 '4567\t6\t1925\n',
 '1925\t3\t50114\n',
 '50114\t6\t1925\n',
 '1925\t3\t98330\n',
 '98330\t6\t1925\n',
 '1925\t3\t77403\n',
 '77403\t6\t1925\n',
 '1925\t3\t93787\n',
 '93787\t6\t1925\n',
 '1925\t3\t138892\n',
 '138892\t6\t1925\n',
 '1925\t3\t71812\n',
 '71812\t6\t1925\n',
 '1925\t3\t119786\n',
 '119786\t6\t1925\n',
 '1925\t3\t103281\n',
 '103281\t6\t1925\n',
 '1925\t3\t55457\n',
 '55457\t6\t1925\n',
 '1925\t3\t81423\n',
 '81423\t6\t1925\n',
 '1925\t3\t71025\n',
 '71025\t6\t1925\n',
 '1925\t3\t31357\n',
 '31357\t6\t1925\n',
 '1925\t3\t78045\n',
 '78045\t6\t1925\n',
 '1925\t3\t110838\n',
 '110838\t6\t1925\n',
 '1925\t3\t91999\n',
 '91999\t6\t1925\n',
 '1925\t3\t29321\n',
 '29321\t6\t1925\n',
 '1925\t3\t27240\n',
 '27240\t6\t1925\n',
 '1925\t3\t124209\n',
 '124209\t6\t1925\n',
 '1925\t3\t107725\n',
 '

In [98]:
path = 'kg_final.txt'
f = open(path, 'w')
f.writelines(lines)
f.close()

In [280]:
sorted_movielens.to_csv("sorted_movielens.csv")

In [165]:
session_lis[-1]

14218

# 測試main裡的所有function功能

In [103]:
import pickle
import math
import time
import random
import torch
import numpy as np
import pandas as pd

train_data = pickle.load(open('./movielens/train.txt', 'rb'))

def split_validation(train_set, valid_portion):
    train_set_x, train_set_y = train_set
    n_samples = len(train_set_x)
    sidx = np.arange(n_samples, dtype='int32')
    np.random.shuffle(sidx)
    n_train = int(np.round(n_samples * (1. - valid_portion)))
    valid_set_x = [train_set_x[s] for s in sidx[n_train:]]
    valid_set_y = [train_set_y[s] for s in sidx[n_train:]]
    train_set_x = [train_set_x[s] for s in sidx[:n_train]]
    train_set_y = [train_set_y[s] for s in sidx[:n_train]]

    return (train_set_x, train_set_y), (valid_set_x, valid_set_y)

train_data, valid_data = split_validation(train_data, 0.1)
test_data = valid_data

# 將知識圖譜的ID從1開始

In [129]:
df_kg = pd.read_csv('.\dataset\kg.txt', sep='\t', header=None, names=['head','relation','tail'])
df_rating = pd.read_csv('.\dataset\\final_movielens.csv', sep=',', names=['itemID', 'rating', 'timestamp', 'sessionID'], skiprows=1)

In [130]:
from sklearn.preprocessing import LabelEncoder
entity_encoder = LabelEncoder()
relation_encoder = LabelEncoder()

In [131]:
entity_encoder.fit(pd.concat([df_rating['itemID'], df_kg['head'], df_kg['tail']]))
relation_encoder.fit(df_kg['relation'])

LabelEncoder()

In [132]:
df_rating['itemID'] = entity_encoder.transform(df_rating['itemID'])
df_kg['head'] = entity_encoder.transform(df_kg['head'])
df_kg['tail'] = entity_encoder.transform(df_kg['tail'])
df_kg['relation'] = relation_encoder.transform(df_kg['relation'])

In [133]:
df_rating['itemID'] = df_rating['itemID']+1
df_kg['head'] = df_kg['head']+1
df_kg['tail'] = df_kg['tail']+1

In [137]:
set(df_kg['tail'])

{16955,
 16956,
 16957,
 16958,
 16959,
 16960,
 16961,
 16962,
 16963,
 16964,
 16965,
 16966,
 16967,
 16968,
 16969,
 16970,
 16971,
 16972,
 16973,
 16974,
 16975,
 16976,
 16977,
 16978,
 16979,
 16980,
 16981,
 16982,
 16983,
 16984,
 16985,
 16986,
 16987,
 16988,
 16989,
 16990,
 16991,
 16992,
 16993,
 16994,
 16995,
 16996,
 16997,
 16998,
 16999,
 17000,
 17001,
 17002,
 17003,
 17004,
 17005,
 17006,
 17007,
 17008,
 17009,
 17010,
 17011,
 17012,
 17013,
 17014,
 17015,
 17016,
 17017,
 17018,
 17019,
 17020,
 17021,
 17022,
 17023,
 17024,
 17025,
 17026,
 17027,
 17028,
 17029,
 17030,
 17031,
 17032,
 17033,
 17034,
 17035,
 17036,
 17037,
 17038,
 17039,
 17040,
 17041,
 17042,
 17043,
 17044,
 17045,
 17046,
 17047,
 17048,
 17049,
 17050,
 17051,
 17052,
 17053,
 17054,
 17055,
 17056,
 17057,
 17058,
 17059,
 17060,
 17061,
 17062,
 17063,
 17064,
 17065,
 17066,
 17067,
 17068,
 17069,
 17070,
 17071,
 17072,
 17073,
 17074,
 17075,
 17076,
 17077,
 17078,
 17079,


In [138]:
np.savetxt(r'.\kg.txt', df_kg.values, fmt='%d')

# KG的檔案必須是單向的，如果是雙向要來用掉

In [178]:
df_kg = pd.read_csv('./dataset/ml-1m/trainning/kg_index.tsv', sep='\t', header=None, names=['head','relation','tail'])
df_kg

,head,relation,tail
0,1925,3,36068
1,36068,6,1925
2,1925,3,10999
3,10999,6,1925
4,1925,3,65992
...,...,...,...
377255,140251,1,2368
377256,140251,9,3118
377257,2995,5,70545
377258,2995,2,63109


In [179]:
df_kg_final = df_kg.query("relation != 5 & relation != 6 & relation != 7 & relation != 8 & relation != 9")

In [180]:
df_kg_final

,head,relation,tail
0,1925,3,36068
2,1925,3,10999
4,1925,3,65992
6,1925,3,138199
8,1925,3,4567
...,...,...,...
377251,102843,1,1158
377253,81063,1,2718
377254,70025,1,3521
377255,140251,1,2368


In [195]:
kg_path = './dataset/ml-1m/trainning/kg_index.tsv'
lines = []
with open(kg_path, 'r') as f:
    for line in f.readlines():
        arr = line.strip().split('\t')
        # arr = line.strip().split(' ')
        head, rel, tail = int(arr[0]), int(arr[1]), int(arr[2])
        if rel==0 or rel==1 or rel==2 or rel==3 or rel==4:
            lines.append(line)

In [196]:
path = 'kg_final.txt'
f = open(path, 'w')
f.writelines(lines)
f.close()

# 將所有ID重整，relation和item都要是從1開始

In [21]:
#如果本身就已經將itemindex轉成entityindex的話 就不需要這一行
#df_item2id = pd.read_csv('item_index2entity_id.txt', sep='\t', header=None, names=['item','id'])
df_kg = pd.read_csv('kg_final.txt', sep='\t', header=None, names=['head','relation','tail'])
df_rating = pd.read_csv('./sorted_movielens.csv', sep=',', names=['itemID', 'rating', 'timestamp', 'sessionID'], skiprows=1)

In [4]:
#只取出有對應entity_id的item
df_rating = df_rating[df_rating['itemID'].isin(df_item2id['item'])]
df_rating.reset_index(inplace=True, drop=True)
df_rating

NameError: name 'df_item2id' is not defined

In [22]:
entity_encoder = LabelEncoder()
relation_encoder = LabelEncoder()

In [23]:
entity_encoder.fit(pd.concat([df_item2id['id'], df_kg['head'], df_kg['tail']]))
relation_encoder.fit(df_kg['relation'])

LabelEncoder()

In [24]:
df_kg['head'] = entity_encoder.transform(df_kg['head'])
df_kg['tail'] = entity_encoder.transform(df_kg['tail'])
df_kg['relation'] = relation_encoder.transform(df_kg['relation'])

In [ ]:
item2id_dict = dict(zip(df_item2id['item'], df_item2id['id']))
item2id_dict

In [ ]:
df_rating['itemID'] = df_rating['itemID'].apply(lambda x: item2id_dict[x])
df_rating['itemID'] = entity_encoder.transform(df_rating['itemID'])

In [201]:
df_rating['itemID'] = df_rating['itemID']+1
df_kg['relation'] = df_kg['relation']
df_kg['head'] = df_kg['head']+1
df_kg['tail'] = df_kg['tail']+1

In [25]:
df_kg

,head,relation,tail
0,1926,3,36069
1,1926,3,11000
2,1926,3,65993
3,1926,3,138200
4,1926,3,4568
...,...,...,...
188618,102844,1,1159
188619,81064,1,2719
188620,70026,1,3522
188621,140252,1,2369


In [26]:
df_kg.to_csv("kg.txt",sep=' ',index=False,header=False)
df_rating.to_csv("final_movielens.csv",index=False)

In [220]:
type(df_kg['tail'].iloc[0])

numpy.int64